In [20]:
import cv2
import numpy as np

# read image
img = cv2.imread('../doorexopen.png')

# read template with alpha
tmplt = cv2.imread('template_deadbolt.png', cv2.IMREAD_UNCHANGED)
hh, ww = tmplt.shape[:2]

# extract template mask as grayscale from alpha channel and make 3 channels
tmplt_mask = tmplt[:,:,3]
tmplt_mask = cv2.merge([tmplt_mask,tmplt_mask,tmplt_mask])

# extract templt2 without alpha channel from tmplt
tmplt2 = tmplt[:,:,0:3]

# do template matching
corrimg = cv2.matchTemplate(img,tmplt2,cv2.TM_CCORR_NORMED, mask=tmplt_mask)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(corrimg)
max_val_ncc = '{:.3f}'.format(max_val)
print("correlation match score: " + max_val_ncc)
xx = max_loc[0]
yy = max_loc[1]
print('xmatch =',xx,'ymatch =',yy)

# draw red bounding box to define match location
result = img.copy()
pt1 = (xx,yy)
pt2 = (xx+ww, yy+hh)
cv2.rectangle(result, pt1, pt2, (0,0,255), 1)

#cv2.imshow('image', img)
#cv2.imshow('template2', tmplt2)
#cv2.imshow('template_mask', tmplt_mask)
cv2.imshow('result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# save results
cv2.imwrite('logo_hat_match2.png', result)

correlation match score: 0.996
xmatch = 466 ymatch = 229


True

(array([247, 248, 248, 248, 249, 249, 249, 249, 249, 250, 250, 250, 250,
       250, 250, 251, 251, 251, 251, 251, 251, 252, 252, 252, 252, 252,
       253, 253, 253, 253, 253, 253, 254, 254, 254, 254, 254, 255, 255,
       255, 255, 255, 256, 256, 256, 256, 256, 257, 257, 257, 257, 257,
       257, 257, 258, 258, 258, 258, 258, 258, 259, 259, 259, 259, 259,
       260, 260, 260, 260, 343, 343, 343, 343, 343, 344, 344, 344, 344,
       344, 344, 344, 345, 345, 345, 345, 345, 345, 345, 345, 346, 346,
       346, 346, 346, 346, 346, 347, 347, 347, 347, 347, 347, 348, 348,
       348, 348, 513, 513, 513, 513, 514, 514, 514, 514, 514, 514, 514,
       514, 515, 515, 515, 515, 515, 515, 515, 515, 515, 516, 516, 516,
       516, 516, 516, 516, 516, 516, 516, 517, 517, 517, 517, 517, 517,
       517, 517, 517, 517, 518, 518, 518, 518, 518, 518, 518, 518, 518,
       518, 519, 519, 519, 519, 519, 519, 519, 519, 519, 520, 520, 520,
       520, 520, 520, 520], dtype=int64), array([339, 338, 339,